### Problem setup from "[An optimization model for aircraft maintenance scheduling and re-assignment](https://www.faa.gov/about/initiatives/maintenance_hf/library/documents/media/aviation_maintenance/science1.pdf)" 


In [0]:
#Import pulp library:
import pulp  
import numpy as np
import random
import itertools
import pandas as pd

##### Parameters

In [0]:
# Definition of constants and variables:
# Definition of airports and routes:
cities = [0,1,2,3,4]
days = [0,1,2,3,4,5,6]
aircrafts = [0,1,2,3,4]
nb_aircrafts = 3
nd_routes = nb_aircrafts

##### Costs and variables

In [0]:
def define_cost_maintenance(aircraft,node):
    ville = node[0]
    jour = node[1]
    cost_A = (1+aircraft/100)*(1+ville/100)#*(1+(jour%4)/100) # maintenance cost depend on aircraft, airport, and day of the week
    cost_B = 1/((2+aircraft/100)*(2+ville/100)  )#*(2+(jour%4)/100))
    return cost_A, cost_B

def define_variable_maintenance(aircraft,node):
    ville = node[0]
    jour = node[1]
    w_cost = pulp.LpVariable("Type A maintenance check node "+str(node)+' aircraft '+str(aircraft),lowBound=0,upBound=1, cat=pulp.LpInteger) 
    Z_cost = pulp.LpVariable("Type B maintenance check node "+str(node)+' aircraft '+str(aircraft),lowBound=0,upBound=1, cat=pulp.LpInteger)
    return w_cost, Z_cost

def define_cost_route(aircraft,node1,node2):
    ville1 = node1[0]
    jour1 = node1[1]
    ville2 = node2[0]
    jour2 = node2[1]
    if ville1 == ville2:
        cost_route = 10000
    else:
        cost_route = 1+ (ville1 - ville2)**2/10 +(1+aircraft/100)
    return cost_route

def define_var_route(aircraft,node1,node2):
    ville1 = node1[0]
    jour1 = node1[1]
    ville2 = node2[0]
    jour2 = node2[1]
    var_route = pulp.LpVariable("Decision var of route "+str(node1)+' '+str(node2)+' for aircraft '+str(aircraft),lowBound=0,upBound=1, cat=pulp.LpInteger)
    return var_route

##### Graph of city-day network

In [0]:
def create_graph(aircraft):
    
    nodes = {
            (city,day): { 'F':[(cityy,(day+1)%len(days)) for cityy in cities],
                          'G':[(cityw,(day-1)%len(days)) for cityw in cities],
                          'Cost_A':define_cost_maintenance(aircraft,(city,day))[0],
                          'Cost_B':define_cost_maintenance(aircraft,(city,day))[1],
                          'variable_A':define_variable_maintenance(aircraft,(city,day))[0],
                          'variable_B':define_variable_maintenance(aircraft,(city,day))[1],
                          
                        } for (city,day) in itertools.product(cities,days)
            }

    edges = {
            ((city_1,(day-1)%len(days)),(city_2,day )): {'Cost_route': define_cost_route( aircraft, (city_1,(day-1)%len(days)),(city_2,day)),
                                              'variable_route':define_var_route( aircraft, (city_1,(day-1)%len(days)),(city_2,day) )
                                             } for (city_1,city_2,day) in itertools.product(cities,cities,days)
            }
    
    return nodes,edges


### Objective function: 
$min \sum_{i=1}^{n_p} \sum_{j=1}^{n_c} \sum_{d=1}^{n_d} ( \underbrace{ g_{ij}w_{ij_d}}_{\textrm{total cost for Type A}} + \underbrace{h_{ij}Z_{ij_d}}_{\textrm{total cost Type B}} ) +  \sum_{i=1}^{n_p}\sum_{ij_{d-1}k_dr \in L}   \underbrace{C_{ij_{d-1}k_dr}.x_{ij_{d-1}k_dr}}_{\textrm{penalty of assigning inappropriate aircraft for the OD trip}}$

cpomdp
GSMDP plus POMDP => belief sur les evenements 

In [0]:
# Problem setup:

prob = pulp.LpProblem("Maintenance Problem, scheduling and allocation", pulp.LpMinimize)

# Objective function:
sum_1 = []
sum_2 = []
nodes = {}
edges = {}

for avion in aircrafts:
    nodes[avion],edges[avion] = create_graph(avion)
    
    sum_1.append([nodes[avion][(ville,jour)]['Cost_A']*nodes[avion][(ville,jour)]['variable_A']  
    + nodes[avion][(ville,jour)]['Cost_B']*nodes[avion][(ville,jour)]['variable_B']
                for ville, jour in nodes[avion].keys()    ])
 
    sum_2.append([edges[avion][(ville_1,jour_1),(ville_2,jour_2)]['Cost_route']*edges[avion][(ville_1,jour_1),(ville_2,jour_2)]['variable_route']   
                for (ville_1,jour_1),(ville_2,jour_2) in edges[avion].keys()  ])
prob += pulp.lpSum(sum_1 + sum_2),"Objective function"

### Constraints definition:
1) An aircraft that comes to city $k$ on day $d$ should leave city $k$ on the next day: 
$	 \sum_{j_{d-1}r}x_{ij_{d-1}k_dr} -  \sum_{j_{d+1}r}x_{ik_dj_{d+1}r} = 0; \textrm{  } j_{d-1}r \in G(k_d),  j_{d+1}r \in F(k_d); \forall i, k_d ; \textrm{ and } d = 1, 2, .. , n_d.$

In [0]:
#Constraint definition:
#1)
for avion in aircrafts:
    for city,day in nodes[avion].keys():
        l_1 = [edges[avion][((city_sum,day_sum),(city,day))]['variable_route'] 
                            for (city_sum,day_sum) in nodes[avion][(city,day)]['G']]
        l_2 = [-edges[avion][((city,day),(city_sum,day_sum))]['variable_route'] 
                      for (city_sum,day_sum) in nodes[avion][(city,day)]['F']]  
        prob += pulp.lpSum(l_1 + l_2)== 0,""

2) An OD trip can be served by one and only one aircraft :

$
\sum_{i=1}^{n_p}x_{ij_{d}k_{d+1}r} = 1; j_{d}k_{d+1}r \in L, d = 1,2, .. n_d.
$

In [0]:
#2) 
# Circumstances changed the problem to an inequality:
for (city_1,day_1),(city_2,day_2) in edges[0].keys():
    prob += pulp.lpSum([edges[avion][((city_1,day_1),(city_2,day_2))]['variable_route'] for avion in aircrafts]) <=1, ""

3) Capacity constraint for Type A check:

$\sum_{i=1}^{n_p} w_{ij_d} \leq p_j, \forall j_d
$

In [0]:
#3)
for (ville,jour) in list(nodes[0].keys()):
    prob += pulp.lpSum( [nodes[avion][(ville,jour)]['variable_A'] for avion in aircrafts] ) <= 5,""

4) Availability constraint of Type A of A/C $i$ at city $j$:
    
$w_{ij_d} - \sum_{k_{d-1}r}x_{ik_{d-1}j_dr} \leq 0;  k_{d-1}r \in G(j_d), \forall j_d
$

In [0]:
#4)
for avion in aircrafts:
    for (city,day) in list(nodes[avion].keys()):
        inter_list = [-edges[avion][((city_sum,day_sum),(city,day))]['variable_route'] 
                           for (city_sum,day_sum) in nodes[avion][(city,day)]['G']]
        inter_list_2 = [nodes[avion][(city,day)]['variable_A']]
        prob += pulp.lpSum(inter_list+inter_list_2) <= 0, ""

5) Ensure that aircraft $i$ undergo Type A check within 4 days:
    
$\sum_{j=1}^{n_c} \sum_{d=m}^{m+3}w_{ij_d} \geq 2; \forall i, m= 1,2, .. n_d
$

In [0]:
#5)
for avion in aircrafts:
    sum_i = []
    for (ville,jour) in list(nodes[0].keys()):
        sum_i.append(pulp.lpSum([nodes[avion][(ville,day)]['variable_A'] for day in range(jour,(jour+3)%len(days))  ]))
    prob += pulp.lpSum(sum_i) >= 2,""

6) Ensure that type B checks are performed once in a given cycle:
    
$\sum_{j_d}Z_{ij_d} =1; j_d \in N, \forall i
$

In [0]:
#6)
for avion in aircrafts:
    prob += pulp.lpSum([nodes[avion][(ville,jour)]['variable_B'] for  (ville,jour) in list(nodes[0].keys())]) ==1,""

7)  Availability constraint of Type B of A/C $i$ at city $j$:
    
$Z_{ij_d} - \sum_{k_{d-1}r}x_{ik_{d-1}j_dr} \leq 0;  k_{d-1}r \in G(j_d), \forall j_d
$

In [0]:
#7)
for avion in aircrafts:
    for city,day in nodes[avion].keys():
        inter_list_b = [-edges[avion][((city_sum,day_sum),(city,day))]['variable_route'] 
                            for (city_sum,day_sum) in nodes[avion][(city,day)]['G']]
        inter_list_b_2 = [nodes[avion][(city,day)]['variable_B'] ]
        prob += pulp.lpSum(inter_list_b+inter_list_b_2) <= 0,""

8) Make sure that the same aircraft is not used for 2 cyclic schedules (the same aircraft shouldn't be assigned to more than one link between 2 consecutive days):
                                                                        
$\sum_{j_2r}x_{ij_{2}k_3r} =1; \forall i \textrm{ and } k(3); j_2r \in G(k_3) 
$

In [0]:
#8)
for aircraft in aircrafts:
    l = [((city_1,day_1),(city_2,day_2)) for (city_1,day_1),(city_2,day_2) in edges[aircraft].keys() if day_1 == 0]
    prob += pulp.lpSum([edges[aircraft][((city_1,day_1),(city_2,day_2))]['variable_route'] for (city_1,day_1),(city_2,day_2) in l]) ==1, ""

### Solution:

In [0]:
%%time
prob.solve()

Wall time: 2.91 s


1

In [0]:
print("Status:", pulp.LpStatus[prob.status])

Status: Optimal


In [0]:
for v in prob.variables():
    if v.varValue ==1:# and v.name[-1] == "1":
        print(v.name, "=", v.varValue)

Decision_var_of_route_(0,_1)_(1,_2)_for_aircraft_1 = 1.0
Decision_var_of_route_(0,_2)_(1,_3)_for_aircraft_4 = 1.0
Decision_var_of_route_(0,_3)_(1,_4)_for_aircraft_0 = 1.0
Decision_var_of_route_(0,_3)_(2,_4)_for_aircraft_2 = 1.0
Decision_var_of_route_(0,_4)_(1,_5)_for_aircraft_3 = 1.0
Decision_var_of_route_(1,_0)_(0,_1)_for_aircraft_1 = 1.0
Decision_var_of_route_(1,_1)_(0,_2)_for_aircraft_4 = 1.0
Decision_var_of_route_(1,_2)_(0,_3)_for_aircraft_2 = 1.0
Decision_var_of_route_(1,_2)_(2,_3)_for_aircraft_1 = 1.0
Decision_var_of_route_(1,_3)_(0,_4)_for_aircraft_3 = 1.0
Decision_var_of_route_(1,_3)_(2,_4)_for_aircraft_4 = 1.0
Decision_var_of_route_(1,_4)_(2,_5)_for_aircraft_0 = 1.0
Decision_var_of_route_(1,_5)_(2,_6)_for_aircraft_3 = 1.0
Decision_var_of_route_(2,_0)_(1,_1)_for_aircraft_4 = 1.0
Decision_var_of_route_(2,_1)_(1,_2)_for_aircraft_2 = 1.0
Decision_var_of_route_(2,_2)_(0,_3)_for_aircraft_0 = 1.0
Decision_var_of_route_(2,_2)_(1,_3)_for_aircraft_3 = 1.0
Decision_var_of_route_(2,_3)_(3